In [1]:
import pandas as pd
import copy
import xlwings as xw
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from data_process import get_csv_files
import warnings
from tqdm import tqdm
import os

In [2]:
warnings.filterwarnings('ignore')
file_path = './data/data'
files = get_csv_files(file_path)

In [3]:
class ChipDistribution():

    def __init__(self):
        self.Chip = {} # 当前获利盘
        self.ChipList = {}  # 所有的获利盘的

    def get_data(self,file):
        self.data = pd.read_csv(file,encoding = "utf-8")
    def calcuJUN(self,dateT,highT, lowT, volT, TurnoverRateT, A, minD):

        x =[]
        l = (highT - lowT) / minD
        for i in range(int(l)):
            x.append(round(lowT + i * minD, 2))
        length = len(x)
        eachV = volT/length
        for i in self.Chip:
            self.Chip[i] = self.Chip[i] *(1 -TurnoverRateT * A)
        for i in x:
            if i in self.Chip:
                self.Chip[i] += eachV *(TurnoverRateT * A)
            else:
                self.Chip[i] = eachV *(TurnoverRateT * A)
        import copy
        self.ChipList[dateT] = copy.deepcopy(self.Chip)



    #按成交量分布计算
    def calcuSin(self,dateT,highT, lowT,avgT, volT,TurnoverRateT,minD,A):
        x =[]

        l = (highT - lowT) / minD
        for i in range(int(l)):
            x.append(round(lowT + i * minD, 2))

        length = len(x)

        #计算仅仅今日的筹码分布
        tmpChip = {}
        eachV = volT/length


        #极限法分割去逼近
        for i in x:
            x1 = i
            x2 = i + minD
            h = 2 / (highT - lowT)
            s= 0
            if i < avgT:
                y1 = h /(avgT - lowT) * (x1 - lowT)
                y2 = h /(avgT - lowT) * (x2 - lowT)
                s = minD *(y1 + y2) /2
                s = s * volT
            else:
                y1 = h /(highT - avgT) *(highT - x1)
                y2 = h /(highT - avgT) *(highT - x2)

                s = minD *(y1 + y2) /2
                s = s * volT
            tmpChip[i] = s


        for i in self.Chip:
            self.Chip[i] = self.Chip[i] *(1 -TurnoverRateT * A)

        for i in tmpChip:
            if i in self.Chip:
                self.Chip[i] += tmpChip[i] *(TurnoverRateT * A)
            else:
                self.Chip[i] = tmpChip[i] *(TurnoverRateT * A)
        
        self.ChipList[dateT] = copy.deepcopy(self.Chip)


    def calcu(self,dateT,highT, lowT,avgT, volT, TurnoverRateT,minD = 0.01, flag=1 , AC=1):
        if flag ==1:
            self.calcuSin(dateT,highT, lowT,avgT, volT, TurnoverRateT,A=AC, minD=minD)
        elif flag ==2:
            self.calcuJUN(dateT,highT, lowT, volT, TurnoverRateT, A=AC, minD=minD)

    def calcuChip(self, flag=1, AC=1):  #flag 使用哪个计算方式,    AC 衰减系数
        low = self.data['low']
        high = self.data['high']
        vol = self.data['volume']
        TurnoverRate = self.data['TurnoverRate']
        avg = self.data['avg']
        date = self.data['trade_date']

        for i in range(len(date)):
        #     if i < 90:
        #         continue

            highT = high[i]
            lowT = low[i]
            volT = vol[i]
            TurnoverRateT = TurnoverRate[i]
            avgT = avg[i]
            # print(date[i])
            dateT = date[i]
            self.calcu(dateT,highT, lowT,avgT, volT, TurnoverRateT/100, flag=flag, AC=AC)  
        # 计算winner
    def winner(self,p=None):
            Profit = []
            date = self.data['trade_date']

            if p == None:  # 不输入默认close
                p = self.data['close']
                count = 0
                for i in self.ChipList:
                    # 计算目前的比例

                    Chip = self.ChipList[i]
                    total = 0
                    be = 0
                    for i in Chip:
                        total += Chip[i]
                        if i < p[count]:
                            be += Chip[i]
                    if total != 0:
                        bili = be / total
                    else:
                        bili = 0
                    count += 1
                    Profit.append(bili)
            else:
                for i in self.ChipList:
                    # 计算目前的比例

                    Chip = self.ChipList[i]
                    total = 0
                    be = 0
                    for i in Chip:
                        total += Chip[i]
                        if i < p:
                            be += Chip[i]
                    if total != 0:
                        bili = be / total
                    else:
                        bili = 0
                    Profit.append(bili)

            #import matplotlib.pyplot as plt
            #plt.plot(date[len(date) - 200:-1], Profit[len(date) - 200:-1])
            #plt.show()

            return Profit

    #计算多少天数据,默认为20
    def lwinner(self,N = 20, p=None):

        data = copy.deepcopy(self.data)
        date = data['trade_date']
        ans = []
        for i in range(len(date)):
            #print(date[i])
            if i < N:
                ans.append(None)
                continue
            self.data = data[i-N:i]
            self.data.index= range(0,N)
            self.__init__()
            self.calcuChip()    #使用默认计算方式
            a = self.winner(p)
            ans.append(a[-1])
        #import matplotlib.pyplot as plt
        #plt.plot(date[len(date) - 100:-1], ans[len(date) - 100:-1])
        #plt.show()

        self.data = data
        return ans



    def cost(self,N):
        date = self.data['trade_date']

        N = N / 100  # 转换成百分比
        ans = []
        for i in self.ChipList:  
            Chip = self.ChipList[i]
            ChipKey = sorted(Chip.keys())  # 排序
            total = 0  # 当前比例
            sumOf = 0  # 所有筹码的总和
            for j in Chip:
                sumOf += Chip[j]

            for j in ChipKey:
                tmp = Chip[j]
                tmp = tmp / sumOf
                total += tmp
                if total > N:
                    ans.append(j)
                    break
        
        #plt.plot(date[len(date) - 1000:-1], ans[len(date) - 1000:-1])
        #plt.show()
        
        return ans
    
    def cal_factor(self,lower_bound,upper_bound):
        factors = {} 
        date = self.data['trade_date']
        for i in self.ChipList.keys(): 
            Chip = self.ChipList[i]
            price_series = self.data.loc[date == i, 'close']
            if price_series.empty:
                continue  # 如果没有找到价格，跳过当前日期
            price = price_series.iloc[0]
            ChipKey = sorted(Chip.keys())  # 排序
            total = 0  
            sumOf = sum(Chip[j] for j in Chip)
            
            for j in ChipKey:
                if (j-price) <= price * upper_bound and (j-price) >= price * lower_bound:
                    total += Chip[j]
            if sumOf != 0:
                factors[i] = total/sumOf
            else:
                if factors:
                    factors[i] = list(factors.values())[-1].copy()
        return factors


In [ ]:

files = get_csv_files(file_path)

def plot_chip(data,file,date,i):
    specific_date = date
    date_data = data[specific_date]

    # 提取价格和对应的值
    prices = list(date_data.keys())
    counts = list(date_data.values())

    # 绘制直方图
    plt.figure(figsize=(6, 3))  # 设置图形大小
    plt.hist(prices, weights=counts, bins=len(prices), alpha=0.75,density=True)

    # 添加标题和标签
    plt.title(f'Distribution on {specific_date}')
    plt.xlabel('Price')
    plt.ylabel('Counts')
    
    img_path = f'{file}.png'
    plt.savefig(img_path)
    plt.close()

    # 尝试打开或创建Excel文件
    excel_file = './data/data/png/2024-07-01.xlsx'
    wb = xw.Book(excel_file)
    sht = wb.sheets['Sheet1']
    # 插入图片到工作表
    left = 10  # 距离单元格左侧的距离
    top = 10   # 距离单元格顶部的距离
    print(img_path)
    try:
        sht.pictures.add(os.path.abspath(img_path), name=f'MyPicture{i+1}', left=sht.range('C5').left + left*100*i, top=sht.range('C5').top + top*i*100, update=True)
        wb.save('./data/data/png/2024-07-01.xlsx')
        wb.close()
    except Exception as e:
        print(f"插入图片时发生错误: {e}")  

    plt.show()
    
for i,file in tqdm(enumerate(files)):    
    a=ChipDistribution()
    a.get_data(file)     
    a.calcuChip(flag=1, AC=1) #计算
    a.winner() #获利盘
    a.cost(90) #成本分布
    a.lwinner(N=1600)
    
    data = a.ChipList
    plot_chip(data,file,'2024-07-01',i)
    

0it [00:00, ?it/s]

./data/data\600008_processed.csv.png


2it [00:59, 32.49s/it]

./data/data\600009_processed.csv.png


3it [01:01, 18.93s/it]

...

41it [07:06, 11.33s/it]

./data/data\600062_processed.csv.png


42it [07:22, 12.53s/it]

./data/data\600063_processed.csv.png


43it [07:29, 10.88s/it]